In [1]:
from APOSR import *
import numpy as np
from tqdm import tqdm
exp_set = get_exp_set(2)

In [2]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from tqdm import tqdm
import itertools
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import numpy as np
from sklearn.model_selection import train_test_split
from pmlb import fetch_data, regression_dataset_names
import time
import statistics as st


def cost(x, xdata, ydata, lambda_string): 
    y_pred = eval(lambda_string)(x, xdata)
    return np.mean(((y_pred - ydata))**2) 

def cust_pred(x, xdata, ydata, lambda_string): 
    y_pred = eval(lambda_string)(x, xdata)
    return y_pred

def random_constant():
        return np.random.randint(1, 31)/10
    
def round_to_n_digits(num, n=3):
    formatstr = '%.'+str(n)+'g'
    return float(formatstr % num)

def list_round_sf(lista, n=3):
    return np.array([round_to_n_digits(i,n) for i in lista])

def permute_movement(lista, n=3):
    rounded_list = list_round_sf(lista,3)
    movement = np.array([round_to_n_digits(i,1) for i in rounded_list*0.01])
    master_list = []
    for idx, i in enumerate(movement):
        temp_list = np.array([0.0]*len(movement))
        temp_list[idx] = i
        master_list.append(list_round_sf(rounded_list+temp_list))
        master_list.append(list_round_sf(rounded_list-temp_list))
    return master_list

def test_SR(string_type="BFGS",divider=3,old_method=False,dataset_name="1027_ESL",random_state=15795):
    
    X, y = fetch_data(dataset_name, return_X_y=True)
    train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=random_state,test_size=0.25)
    holdout_X, test_X, holdout_y, test_y = train_test_split(test_X, test_y, random_state=random_state,test_size=0.6)
    shuffle_idx = np.random.permutation(len(train_y))
    train_X = train_X[shuffle_idx]
    train_y = train_y[shuffle_idx]
    X_train = np.array(train_X)
    y_train = np.array(train_y)
    X_test = test_X
    y_test = test_y
    
    master_list=[]
    master_list2=[]
    xdata = X_train.T
    ydata = y_train
    xvalid = X_test.T
    yvaliddata= y_test
    holdout_X = holdout_X.T
    holdout_y = holdout_y
    
    num_of_feature = xdata.shape[0]
    
    for i in range(10):
        mse_tuple = tuple()

        for test_eq in tqdm(exp_set):
            test_eq_orig = test_eq
            R_count = test_eq.count("R")
            
            lambda_string = "lambda x,xdata:"
            
            if old_method:
                possible_combi_of_num = np.array(
                    list(itertools.product(range(num_of_feature + 1), repeat=R_count))
                )
                if len(possible_combi_of_num) > 100:
                    possible_combi_of_num = possible_combi_of_num[
                        np.random.choice(len(possible_combi_of_num), 100, replace=False)
                    ]
                for combi_var in itertools.product(range(num_of_feature+1), repeat=R_count):
                    lambda_string = "lambda x,xdata:"
                    test_eq=test_eq_orig
                    index = 0
                    for i in combi_var:
                        if i==num_of_feature:
                            test_eq = test_eq.replace("R", f"x[{index}]", 1)
                            index+=1
                        else:
                            test_eq = test_eq.replace("R", f"xdata[{i}]", 1)
                    lambda_string += test_eq
                    try:
                        res = minimize(cost,
                                       x0 = [random_constant() for i in range(index+1)],
                                       args=(xdata,ydata,lambda_string),
                                       method=string_type,options={"maxiter":150})
                        optimized_cost = cost(res.x, xdata, ydata, lambda_string)
                        master_list.append((test_eq_orig,test_eq,res.x,res.nit,optimized_cost))
                    except RuntimeError:
                        print("No fit found")
            else:
                for i in range(R_count):
                    test_eq = test_eq.replace("R", f"x[{i*(num_of_feature+1)}]"+"".join([f"+x[{i*(num_of_feature+1)+1+j}]*xdata[{j}]" for j in range(num_of_feature)]), 1)
                lambda_string += test_eq
                try:
                    start_time = time.process_time()
                    x0 = [random_constant() for i in range(R_count*(xdata.shape[0]+1))]
                    num_rounds = 5
                    epoch_size= len(ydata)//(num_rounds+1)
                    max_rounds = 2
                    for j in range(max_rounds):                            
                        for i in range(num_rounds if num_rounds else 1):
                            res = minimize(cost,
                                           x0 = x0,
                                           args=(xdata[:,i*epoch_size:(i+1)*epoch_size],ydata[i*epoch_size:(i+1)*epoch_size],lambda_string),
                                           method=string_type,options={"maxiter":10})
                            x0=res.x
                            num_of_neurons = len(x0)//(xdata.shape[0]+1)
                            for i in range(num_of_neurons):
                                temp_arr = np.repeat([x0[i*(xdata.shape[0]+1):(i+1)*(xdata.shape[0]+1)]],xdata.shape[1], axis=0)
                                temp_arr = np.multiply(temp_arr[:,1:],xdata.T)
                                sum_arr = np.sum(temp_arr,axis=1)
                                diff_arr = temp_arr - np.repeat([sum_arr],xdata.shape[0],axis=0).T
                                diff_arr = np.abs(diff_arr)
                                best_idx = st.mode(np.argmin(diff_arr, axis=1))+1
                                old_x0 = x0.copy()
                                x0[i*(xdata.shape[0]+1):(i+1)*(xdata.shape[0]+1)] = 0 #set all to 0
                                x0[i*(xdata.shape[0]+1)]=old_x0[i*(xdata.shape[0]+1)] # retain bias
                                x0[i*(xdata.shape[0]+1)+best_idx]=old_x0[i*(xdata.shape[0]+1)+best_idx] # retain best weight
                            if time.process_time()-start_time >10:
                                print("timeout")
                                break
                        if j == max_rounds-2:#second last round
                            new_x0=list()
                            for i in range(num_of_neurons):
                                temp_arr = np.repeat([x0[i*(xdata.shape[0]+1):(i+1)*(xdata.shape[0]+1)]],xdata.shape[1], axis=0)
                                temp_arr = np.multiply(temp_arr[:,1:],xdata.T)
                                sum_arr = np.sum(temp_arr,axis=1)
                                diff_arr = temp_arr - np.repeat([sum_arr],xdata.shape[0],axis=0).T
                                diff_arr = np.abs(diff_arr)
                                best_idx = st.mode(np.argmin(diff_arr, axis=1))+1
                                old_x0 = x0.copy()
                                x0[i*(xdata.shape[0]+1):(i+1)*(xdata.shape[0]+1)] = 0 #set all to 0
                                x0[i*(xdata.shape[0]+1)]=old_x0[i*(xdata.shape[0]+1)] # retain bias
                                x0[i*(xdata.shape[0]+1)+best_idx]=old_x0[i*(xdata.shape[0]+1)+best_idx] # retain best weight
                                for uninfluential_index in range(i*(xdata.shape[0]+1),(i+1)*(xdata.shape[0]+1)):
                                    if uninfluential_index==i*(xdata.shape[0]+1)+best_idx:
                                        new_x0.append(x0[i*(xdata.shape[0]+1)+best_idx])
                                        lambda_string = lambda_string.replace(f"x[{uninfluential_index}]",f"x[{i*2+1}]")
                                    elif uninfluential_index==i*(xdata.shape[0]+1):
                                        new_x0.append(x0[i*(xdata.shape[0]+1)])
                                        lambda_string = lambda_string.replace(f"x[{uninfluential_index}]",f"x[{i*2}]")
                                    else:
                                        lambda_string = lambda_string.replace(f"x[{uninfluential_index}]",f"0")
                            x0 = np.array(new_x0)
                            
                            
                    optimized_cost = cost(res.x, xdata, ydata, lambda_string)
                    ypredtrain = cust_pred(res.x, xdata, ydata, lambda_string)
                    ypred = cust_pred(res.x, xvalid, yvaliddata, lambda_string)
                    y_holdout_pred = cust_pred(res.x, holdout_X, holdout_y, lambda_string)
                    
                    if sum(np.isnan(ypred)) or sum(np.isinf(ypred)) or sum(np.isnan(ypredtrain)) or sum(np.isinf(ypredtrain)) or sum(np.isnan(y_holdout_pred)) or sum(np.isinf(y_holdout_pred)):
                        master_list.append((test_eq_orig,lambda_string,res.x,res.nit,optimized_cost,float('inf'),float('inf'),float('inf'),float('inf'),float('inf'),float('inf'),float('inf') ) ) 
                    else:
                        master_list.append((test_eq_orig,lambda_string,res.x,res.nit,mean_squared_error(ydata,ypredtrain),mean_squared_error(yvaliddata,ypred),mean_squared_error(holdout_y,y_holdout_pred),
                                            mean_absolute_error(ydata,ypredtrain),mean_absolute_error(yvaliddata,ypred),mean_absolute_error(holdout_y,y_holdout_pred),
                                            r2_score(ydata,ypredtrain),r2_score(yvaliddata,ypred),r2_score(holdout_y,y_holdout_pred),dataset_name,random_state ) )
                    
                except RuntimeError:
                    print("No fit found")
                    
    best_func=sorted(master_list, key=lambda x: x[5])
    guess_x0 = list_round_sf(np.array(best_func[0][2]))
    lambda_string = best_func[0][1]
    best_x0 = guess_x0.copy()
    best_cost = cost(best_x0, xdata, ydata, lambda_string)
    changed = True
    counter = 0
    while changed and counter<100:        
        changed = False
        for test_x0 in permute_movement(best_x0):
            cur_cost = cost(test_x0, xdata, ydata, lambda_string)
            if cur_cost<best_cost:
                best_x0 = test_x0.copy()
                best_cost = cur_cost
                changed = True
        #print(best_x0)
        #print(cost(best_x0, xdata, ydata, lambda_string))
        counter+=1
        
    optimized_cost = cost(best_x0, xdata, ydata, lambda_string)
    ypredtrain = cust_pred(best_x0, xdata, ydata, lambda_string)
    ypred = cust_pred(best_x0, xvalid, yvaliddata, lambda_string)
    y_holdout_pred = cust_pred(best_x0, holdout_X, holdout_y, lambda_string)
                  
    if sum(np.isnan(ypred)) or sum(np.isinf(ypred)) or sum(np.isnan(ypredtrain)) or sum(np.isinf(ypredtrain)) or sum(np.isnan(y_holdout_pred)) or sum(np.isinf(y_holdout_pred)):
        master_list2.append((test_eq_orig,lambda_string,best_x0,res.nit,optimized_cost,float('inf'),float('inf'),float('inf'),float('inf'),float('inf'),float('inf'),float('inf') ) ) 
    else:
        master_list2.append((test_eq_orig,lambda_string,best_x0,res.nit,mean_squared_error(ydata,ypredtrain),mean_squared_error(yvaliddata,ypred),mean_squared_error(holdout_y,y_holdout_pred),
                            mean_absolute_error(ydata,ypredtrain),mean_absolute_error(yvaliddata,ypred),mean_absolute_error(holdout_y,y_holdout_pred),
                            r2_score(ydata,ypredtrain),r2_score(yvaliddata,ypred),r2_score(holdout_y,y_holdout_pred),dataset_name,random_state ) )
                
    return master_list, master_list2

NumExpr defaulting to 8 threads.


In [3]:
APOSR_solution, _ = test_SR(dataset_name="1027_ESL")
print(f"R2_Score: {sorted(APOSR_solution, key=lambda x: x[5])[0][-3]}")

100%|██████████| 31/31 [00:10<00:00,  2.94it/s]


R2_Score: 0.8625596983255885
